In [112]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm

In [113]:
BATCH_SIZE = 16
EMBED_SIZE = 510
BLOCK_SIZE = 128
HEAD_SIZE = 6
EPOCHS = 5000
EPOCHS_VAL = 200
N_GRAMM = 3

In [114]:
with open('./war_and_peace.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [115]:
print(text[:1000])

Лев Николаевич Толстой
Война и мир. Книга 1

Война и мир – 1

Аннотация 

Роман Льва Толстого «Война и мир» лежит в основании величественного здания русской классической литературы. С непревзойденным мастерством Толстой воссоздал великую духом Россию – образы этой «книги на все времена» и сейчас пленяют свежестью чувств и щедростью души, искренностью страстей, силой и чистотой убеждений.
В книгу вошли первый и второй тома романа.

Лев Николаевич Толстой
ВОЙНА И МИР

Том 1

ЧАСТЬ ПЕРВАЯ


I

– Еh bien, mon prince. Genes et Lucques ne sont plus que des apanages, des поместья, de la famille Buonaparte. Non, je vous previens, que si vous ne me dites pas, que nous avons la guerre, si vous vous permettez encore de pallier toutes les infamies, toutes les atrocites de cet Antichrist (ma parole, j'y crois) – je ne vous connais plus, vous n'etes plus mon ami, vous n'etes plus мой верный раб, comme vous dites. [Ну, что, князь, Генуа и Лукка стали не больше, как поместьями фамилии Бонапарте. Нет, 

In [116]:
def get_ngramms(n, text):
    vocab = set([])
    for i in range(len(text)-n):
        vocab.add(text[i:i+n])
    
    return sorted(list(vocab))

In [117]:
ngramms = get_ngramms(n=N_GRAMM, text=text)
VOCAB_SIZE = len(ngramms)
print(ngramms)
print(VOCAB_SIZE)

['\n\n\n', '\n\n1', '\n\n3', '\n\nI', '\n\nV', '\n\nX', '\n\n«', '\n\nА', '\n\nБ', '\n\nВ', '\n\nГ', '\n\nД', '\n\nЕ', '\n\nЖ', '\n\nЗ', '\n\nИ', '\n\nК', '\n\nЛ', '\n\nМ', '\n\nН', '\n\nО', '\n\nП', '\n\nР', '\n\nС', '\n\nТ', '\n\nУ', '\n\nХ', '\n\nЧ', '\n\n–', '\n\n…', '\n(В', '\n(Э', '\n1)', '\n11', '\n12', '\n13', '\n18', '\n2)', '\n24', '\n26', '\n28', '\n3 ', '\n3)', '\n31', '\n4 ', '\n4)', '\nAu', '\nDa', '\nI\n', '\nII', '\nIV', '\nIX', "\nL'", '\nLe', '\nLi', '\nM ', '\nOn', '\nPS', '\nV\n', '\nVI', '\nVi', '\nX\n', '\nXI', '\nXV', '\nXX', '\n[В', '\n[Е', '\n[М', '\n[П', '\n[С', '\n[Я', '\n«2', '\n«3', '\n«7', '\n«9', '\n«A', '\n«B', '\n«C', '\n«D', '\n«E', '\n«I', '\n«J', '\n«L', '\n«M', '\n«P', '\n«Q', '\n«S', '\n«T', '\n«V', '\n«d', '\n«А', '\n«Б', '\n«В', '\n«Г', '\n«Д', '\n«Е', '\n«З', '\n«И', '\n«К', '\n«Л', '\n«М', '\n«Н', '\n«О', '\n«П', '\n«Р', '\n«С', '\n«Т', '\n«У', '\n«Х', '\n«Ч', '\n«Ш', '\n«Э', '\n«Я', '\nА ', '\nАв', '\nАд', '\nАк', '\nАл', '\nАн', '\nАр', '\nАт

In [118]:
stoi = {ch:i for i, ch in enumerate(ngramms)}
itos = {i:ch for i, ch in enumerate(ngramms)}

def encode(s):
    result = []
    for i in range(0, len(s)-N_GRAMM, N_GRAMM):
        result.append(stoi[s[i:i+N_GRAMM]])
    return result

decode = lambda l: ''.join([itos[i] for i in l])

In [119]:
print(encode('Привет, как дела?'))
print(decode(encode('Привет, как дела?')))

[7841, 9444, 2252, 8624, 10063]
Привет, как дел


In [120]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:1000])

torch.Size([488940])
tensor([ 7521,  1088, 12494,  8541, 12008,  8051, 15675, 12089, 14233,  8381,
         1462, 14884,  7476,  9760,  2483,  7120, 13506, 11544, 11888, 18996,
            7, 13675, 15942, 12060,    22, 14296, 13441, 18190,  1168, 12891,
        14093,   861, 14233,  8381,  1462, 14898, 12750, 11937,  9332, 15589,
         9390, 11734,  9439, 12017, 15667, 10713, 14093,  1392,  8716, 18650,
        16649, 12492,  1434,  8814, 12017, 15563, 12141, 11948, 14926, 16631,
         2386,  1474, 14752,  9459, 12196, 13677, 13110,  8815, 10785, 15961,
        13093, 14275, 16092,  1315, 15648, 11280, 12629, 10653, 12570,  1351,
        17053,  1133, 15643, 18447,  1487, 14915, 17804, 16092,   890, 13631,
        11545,  8375, 15488,  9561, 13217,  8442, 11549, 10615,  8794, 14700,
        13879, 15867,  9434, 10813, 18301, 17384, 15667,  1402, 17672, 15137,
        16248,  1351, 17526,  1416, 12509, 13675, 15680, 18435, 16120, 15669,
        12172, 15536, 14218, 11555, 11927, 

In [121]:
split_idx = int(0.9*len(data))
train_data = data[:split_idx]
test_data = data[split_idx:]

In [122]:
def get_batch(split):
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
    return x, y


xb, yb = get_batch('train')
print('Input:')
print(xb)

print('Target:')
print(yb)



Input:
tensor([[ 8636, 16920, 14429,  ..., 16074,  9443, 10656],
        [ 1429, 12282, 14391,  ..., 11856,  1318, 12748],
        [10018, 13506, 15476,  ..., 11960,  1555, 14218],
        ...,
        [14536, 12644,  8728,  ...,  9481, 15782, 11549],
        [ 6358, 13303,  1307,  ..., 11948,  2430,  7686],
        [14442,  9247,  1436,  ...,  1591,  9861,  1655]])
Target:
tensor([[16920, 14429, 14299,  ...,  9443, 10656, 10359],
        [12282, 14391, 15389,  ...,  1318, 12748,  1390],
        [13506, 15476, 11802,  ...,  1555, 14218, 14472],
        ...,
        [12644,  8728, 14054,  ..., 15782, 11549, 15933],
        [13303,  1307, 15113,  ...,  2430,  7686,  1392],
        [ 9247,  1436, 13180,  ...,  9861,  1655, 13940]])


In [123]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(EPOCHS_VAL)
        for k in range(EPOCHS_VAL):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [124]:
class Head(nn.Module):
    def __init__(self, head_size):
        super(Head, self).__init__()
        self.key = nn.Linear(EMBED_SIZE, head_size, bias=False)
        self.query = nn.Linear(EMBED_SIZE, head_size, bias=False)
        self.value = nn.Linear(EMBED_SIZE, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(BLOCK_SIZE, BLOCK_SIZE)))
        self.dropout = nn.Dropout(0.2)

    
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        w = q @ k.transpose(1, 2) * C**-0.5
        w = w.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        w = F.softmax(w, dim=-1)
        w = self.dropout(w)
        v = self.value(x)
        out = w @ v
        return out

In [125]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super(MultiHeadAttention, self).__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(EMBED_SIZE, EMBED_SIZE)
        self.dropout = nn.Dropout(0.2)

    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        return self.dropout(self.proj(out))

In [126]:
class FFN(nn.Module):
    def __init__(self, embed_size):
        super(FFN, self).__init__()
        self.ffn = nn.Sequential(
            nn.Linear(embed_size, embed_size*4),
            nn.ReLU(),
            nn.Linear(embed_size*4, embed_size),
            nn.Dropout(0.2)
        )

    
    def forward(self, x):
        return self.ffn(x)

In [127]:
class Block(nn.Module):
    def __init__(self, embed_size, head_size):
        super(Block, self).__init__()
        h_size = embed_size // head_size
        self.sa = MultiHeadAttention(head_size, h_size)
        self.ffn = FFN(embed_size)
        self.ln1 = nn.LayerNorm(embed_size)
        self.ln2 = nn.LayerNorm(embed_size)

    
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffn(self.ln2(x))
        return(x)

In [128]:
class LM(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embed = nn.Embedding(VOCAB_SIZE, EMBED_SIZE)
        self.pos_embed = nn.Embedding(BLOCK_SIZE, EMBED_SIZE)
        self.blocks = nn.Sequential(*[Block(EMBED_SIZE, HEAD_SIZE) for _ in range(6)])
        self.ln = nn.LayerNorm(EMBED_SIZE)
        self.lm_head = nn.Linear(EMBED_SIZE, VOCAB_SIZE)
        

    def forward(self, x, y=None):
        B, T = x.shape
        tok_embed = self.token_embed(x)
        pos_embed = self.pos_embed(torch.arange(T))
        out = tok_embed + pos_embed
        out = self.blocks(out)
        out = self.ln(out)
        out = self.lm_head(out)
        
        if y is None:
            loss = None
        else:
            B, T, C = out.shape
            out = out.view(B*T, C)
            y = y.view(B*T)
            loss = F.cross_entropy(out, y)

        return out, loss


    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02) 

    
    def generate(self, x, max_new_tokens):
        for _ in range(max_new_tokens):
            x_cond = x[:, -BLOCK_SIZE:]
            out, loss = self(x_cond)
            out = out[:, -1, :]
            probs = F.softmax(out, dim=-1)
            x_next = torch.multinomial(probs, num_samples=1)
            x = torch.cat((x, x_next), dim=1)
        return x

In [129]:
model = LM()
out, loss = model(xb, yb)
print(out.shape)
print(loss.item())

torch.Size([2048, 19235])
10.031195640563965


In [130]:
best_model = None
best_loss = float('inf')

In [131]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
for epoch in tqdm(range(EPOCHS)):
    xb, yb = get_batch('train')

    out, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if epoch % 500 == 0:
        losses = estimate_loss()
        print(f"step {epoch}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if losses['val'] < best_loss:
            best_loss = losses['val']
            best_model = model.state_dict()

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 1/5000 [07:34<630:37:41, 454.14s/it]

step 0: train loss 9.8888, val loss 9.8769


 10%|█         | 501/5000 [43:28<165:39:25, 132.56s/it]

step 500: train loss 5.2291, val loss 5.4963


 20%|██        | 1001/5000 [1:22:44<189:12:49, 170.33s/it]

step 1000: train loss 4.4876, val loss 5.1267


 21%|██        | 1059/5000 [1:26:49<5:23:05,  4.92s/it]   


KeyboardInterrupt: 

In [133]:
x = torch.zeros((1, 1), dtype=torch.long)
generated_text = decode(model.generate(x, max_new_tokens=)[0].tolist())
generated_text

'\n\n\ne ne vous– Шу Марьныйктал 1 toe… кли, [Ну он с ребе. – В гордились, – сказала ch ю.\nОдина, голосами красным княгиня, но наподко, низte …] цкой в Австродутил князь Андрей. Обепымши во лицом союарфгу приезжого Андрея. Он сде его рассказалось крайнем рассказала.\n– Прежду счасрез подняла Марьи hanбже отед, под ренского будешения на то,нюшула полковнился, пром, обгомой огрому седы в которую он нам ошо, аа Павление. Дядюшки и лошадь.\n– Николай, помоi pлавноковнул в Москорошо»ие. И развершаю meшей первосебя в этоговорил Пьера было офицер, – А то времилий Д думал лицо у ме и весных в Ростову, вот что полдаря ему голосом.\nВдруг вство, прежнего пожиму савра, и за гые,ся! Он слица в слов, туя, и умеезрубяться и мы его единство с чперед ости были на еще можный разрешальцы счастные стова, оченькую которым выражение, принял быть в своих мере одил пита, что, что жела для жавить решительностинод милу и только гостичестветельно улыбки вфиней засмеялись повеческих и упал сbleнь]ns л цю извее 

In [ ]:
with open('generated_text.txt', 'w', encoding='utf-8') as f:
    f.write(generated_text)

In [ ]:
# torch.save(model.state_dict(), 'war_and_peace_transformer.pth')